In [4]:
from pyscf import gto,scf
import pandas as pd
from pyscf.hessian import rhf as hrhf
import numpy as np 
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
import basis_set_exchange as bse
import sys
dl=.05
sys.path.append('../Libs/')
from FDcoeffs import *
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM

In [5]:
mol=gto.M(atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))

In [6]:
hfCO=scf.RHF(mol)
hfCO.scf()

converged SCF energy = -112.786616218695


-112.78661621869524

In [7]:
dl=.1
co_gs=[]
co_hs=[]
co_amds=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,-i*dl],atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_1e())
    g=mf.Gradients()
    grad=g.grad()
    h=mf.Hessian()
    co_hs.append(h.hess()[1,1,2,2])
    co_gs.append(grad[1,2])
    co_amds.append((aaff(mf,[1,-1])+alc_deriv_grad_nuc(fm,[1,-1]))[1,2])

converged SCF energy = -116.424997572095
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     0.1002509309
1 O    -0.0000000000    -0.0000000000    -0.1002509309
----------------------------------------------
converged SCF energy = -115.402676804101
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.0735561124
1 O    -0.0000000000     0.0000000000    -0.0735561124
----------------------------------------------
converged SCF energy = -114.455410086041
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000    -0.0000000000     0.0478011505
1 O     0.0000000000     0.0000000000    -0.0478011505
----------------------------------------------
converged SCF energy = -113.583344282024
--------------- RHF gradients ---------------
         x                y                z
0

In [8]:
co_gs=np.asarray(co_gs)[:,1,2]
co_hs=np.asarray(co_hs)[:,1,1,2,2]
co_amds=np.asarray(co_amds)[:,1,2]

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [ ]:
np.save("CO_grads_9pfd",co_gs),np.save("CO_hess_9pfd",co_hs)
np.save("CO_aderivs_9pfd",co_amds)

In [ ]:
dl=.1
bf_gs=[]
bf_hs=[]
bf_amds=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,-i*dl],atom="B 0 0 0; F 0 0 2.35348",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_1e(),max_cycle=100)
    g=mf.Gradients()
    h=mf.Hessian()
    bf_hs.append(h.hess()[1,1,2,2])
    grad=g.grad()
    bf_gs.append(grad[1,2])
    bf_amds.append(aaff(mf,[1,-1])+alc_deriv_grad_nuc(fm,[1,-1])) 

In [ ]:
bf_gs=np.asarray(bf_gs)[:,1,2]
bf_hs=np.asarray(bf_hs)[:,1,1,2,2]
bf_amds=np.asarray(bf_amds)[:,1,2]

In [ ]:
np.save("BF_grads_9pfd",bf_gs),np.save("BF_hess_9pfd",bf_hs)
np.save("BF_aderivs_9pfd",bf_amds)

In [ ]:
dl=.1
nn_gs=[]
nn_hs=[]
nn_amds=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,-i*dl],atom="N 0 0 0; N 0 0 2.01388999",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_1e(),max_cycle=100)
    g=mf.Gradients()
    h=mf.Hessian()
    nn_hs.append(h.hess()[1,1,2,2])
    grad=g.grad()
    nn_gs.append(grad[1,2])
    nn_amds.append((aaff(mf,[1,-1])+alc_deriv_grad_nuc(fm,[1,-1]))[1,2])

In [ ]:
np.save("NN_grads_9pfd",nn_gs),np.save("NN_hess_9pfd",nn_hs)
np.save("NN_aderivs_9pfd",nn_amds)

In [ ]:
#CO
print("1 order finite diff. accuracy:: ",(co_gs[5]-co_gs[4])/dl)
print("2 order finite diff. accuracy:: ",np.dot(fd1_2,co_gs)/dl)
print("4 order finite diff. accuracy:: ",np.dot(fd1_4,(np.asarray(co_gs)))/dl)
print("6 order finite diff. accuracy:: ",np.dot(fd1_6,(np.asarray(co_gs)))/dl)
print("8 order finite diff. accuracy:: ",np.dot(fd1_8,(np.asarray(co_gs)))/dl)
print("Analytical derivative        :: ",co_amds[4])

In [ ]:
#BF
print("1 order finite diff. accuracy:: ",(bf_gs[5]-bf_gs[4])/dl)
print("2 order finite diff. accuracy:: ",np.dot(fd1_2,bf_gs)/dl)
print("4 order finite diff. accuracy:: ",np.dot(fd1_4,bf_gs)/dl)
print("6 order finite diff. accuracy:: ",np.dot(fd1_6,bf_gs)/dl)
#print("8 order finite diff. accuracy:: ",np.einsum('i,ijk->jk',fd1_8,(np.asarray(bf_ge)))[1,2]/dl)
print("Analytical derivative        :: ",bf_amds[4])

In [ ]:
#2 order finite diff. accuracy::  0.266513
#4 order finite diff. accuracy::  0.266778
#6 order finite diff. accuracy::  0.266777
#Analytical derivative        ::  0.266777

In [ ]:
#with dl=.1  the 4th derivatives are:   only McL expansion of aaff is consistent to obtain APDFT4_g1 derivs
#[[-5.43974711e-09 -4.46784523e-09 -8.97038551e-02]
# [ 5.43974711e-09  4.46784523e-09  8.97038538e-02]]
#[[ 1.04025962e-09  1.59691820e-09 -1.55567761e-01]
# [-2.70775954e-08  7.58253933e-09  1.56759171e-01]]

In [ ]:
np.save("BF_grads_9pfd",bf_gs),np.save("BF_hess_9pfd",bf_hs)
np.save("BF_aderivs_9pfd",bf_amds)

# Target Gradients / Hessians 

In [ ]:
BF_CO=FcM(fcs=[-1,1],atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
hfBF_CO=scf.RHF(BF_CO)
hfBF_CO.scf(dm0=hfBF_CO.init_guess_by_1e())
geBF_CO=hfBF_CO.Gradients().grad_elec()
gBF_CO=geBF_CO+hfBF_CO.Gradients().grad_nuc()
hBF_CO=hfBF_CO.Hessian().hess()

In [ ]:
NN_CO=FcM(fcs=[1,-1],atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
hfNN_CO=scf.RHF(NN_CO)
hfNN_CO.scf(dm0=hfNN_CO.init_guess_by_1e())
geNN_CO=hfNN_CO.Gradients().grad_elec()
gNN_CO=geNN_CO+hfNN_CO.Gradients().grad_nuc()
hNN_CO=hfNN_CO.Hessian().hess()

In [ ]:
CO_NN=FcM(fcs=[-1,1],atom="N 0 0 0; N 0 0 2.01388999",unit="Bohrs",\
          basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
hfCO_NN=scf.RHF(CO_NN)
hfCO_NN.scf(dm0=hfCO_NN.init_guess_by_1e())
geCO_NN=hfCO_NN.Gradients().grad_elec()
gCO_NN=geCO_NN+hfCO_NN.Gradients().grad_nuc()
hCO_NN=hfCO_NN.Hessian().hess()

In [ ]:
co_bf=FcM(fcs=[1,-1],atom="B 0 0 0; F 0 0 2.35348",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
hfco_bf=scf.RHF(co_bf)
hfco_bf.scf(dm0=hfco_bf.init_guess_by_1e())
geco_bf=hfco_bf.Gradients().grad_elec()
gco_bf=geco_bf+hfCO_NN.Gradients().grad_nuc()
hco_bf=hfco_bf.Hessian().hess()

In [ ]:
hco_bf.shape

In [ ]:
CO=gto.M(atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",\
       basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
NN=gto.M(atom="N 0 0 0; N 0 0 2.01388999",unit="Bohrs",\
          basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
BF=gto.M(atom="B 0 0 0; F 0 0 2.35348",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))

In [ ]:
and1CO,and2CO=alc_deriv_grad_nuc(CO,[1,-1]),alc_differential_grad_nuc(CO,[1,-1])-alc_deriv_grad_nuc(CO,[1,-1])
and1NN,and2NN=alc_deriv_grad_nuc(NN,[-1,1]),alc_differential_grad_nuc(NN,[-1,1])-alc_deriv_grad_nuc(NN,[-1,1])
and1BF,and2BF=alc_deriv_grad_nuc(BF,[1,-1]),alc_differential_grad_nuc(BF,[1,-1])-alc_deriv_grad_nuc(BF,[1,-1])

In [ ]:
q=[]
q.append({"alchemy":"CO@BF","ge":geco_bf[1,2], "grad":gco_bf[1,2],"hess":hco_bf[1,1,2,2],"and1":and1BF[1,2] ,"and2": and2BF[1,2]})
q.append({"alchemy":"CO@NN","ge":geCO_NN[1,2], "grad":gCO_NN[1,2],"hess":hCO_NN[1,1,2,2],"and1":and1NN[1,2] ,"and2":  and2NN[1,2]})
q.append({"alchemy":"BF@CO","ge":geBF_CO[1,2], "grad":gBF_CO[1,2],"hess":hBF_CO[1,1,2,2],"and1":-and1CO[1,2],"and2": and2CO[1,2] })
q.append({"alchemy":"NN@CO","ge":geNN_CO[1,2], "grad":gNN_CO[1,2],"hess":hNN_CO[1,1,2,2],"and1":and1CO[1,2] ,"and2": and2CO[1,2] })
pd.DataFrame(q).to_pickle("alch_transf_mol")

In [ ]:
hfBF.Gradients().grad_nuc()+ge[4]+amds[4]+np.einsum('i,ijk->jk',fd1_8,(np.asarray(amds)))/dl/2  \
    +np.einsum('i,ijk->jk',fd2_8,(np.asarray(amds)))/dl**2/6+np.einsum('i,ijk->jk',fd3_6,(np.asarray(amds)))/dl**3/24 \
  +np.einsum('i,ijk->jk',fd4_6,(np.asarray(amds)))/dl**4/120+np.einsum('i,ijk->jk',fd5_4,(np.asarray(amds)))/dl**5/720 \
 +np.einsum('i,ijk->jk',fd6_2,(np.asarray(amds)))/dl**6/5040

In [ ]:
#hfBF.Gradients().grad_nuc()+ge[4]+amds[4]/2,amds[4]

In [ ]:
(hfBF.Gradients().grad_nuc()-hfCO.Gradients().grad_nuc())/(-2)

In [ ]:
amds=co_amds

In [ ]:
aaff_predict=[]
aaff_predict.append((hfCO.Gradients().grad_nuc()+ge[4])[1,2])
aaff_predict.append(aaff_predict[0]+alc_deriv_grad_nuc(mol,[-1,1])[1,2]+amds[4][1,2])
aaff_predict.append(aaff_predict[1]+alc_differential_grad_nuc(mol,[-1,1])[1,2]\
                    -alc_deriv_grad_nuc(mol,[-1,1])[1,2]+np.einsum('i,ijk->jk',fd1_8,(np.asarray(amds)))[1,2]/dl/2)
aaff_predict.append(aaff_predict[2]+np.einsum('i,ijk->jk',fd2_8,(np.asarray(amds)))[1,2]/dl**2/6)
aaff_predict.append(aaff_predict[3]+np.einsum('i,ijk->jk',fd3_6,(np.asarray(amds)))[1,2]/dl**3/24 )
aaff_predict.append(aaff_predict[4]+np.einsum('i,ijk->jk',fd4_4,(np.asarray(amds)))[1,2]/dl**4/120)
aaff_predict.append(aaff_predict[5]+np.einsum('i,ijk->jk',fd5_4,(np.asarray(amds)))[1,2]/dl**5/720)
aaff_predict.append(aaff_predict[6]+np.einsum('i,ijk->jk',fd6_2,(np.asarray(amds)))[1,2]/dl**6/5040)
np.asarray(aaff_predict),np.asarray(aaff_predict)-(hfBF.Gradients().grad_nuc()+gBF)[1,2]

In [ ]:
plt.plot(range(1,8),abs(np.asarray(aaff_predict)-(hfBF.Gradients().grad_nuc()+gBF)[1,2])[1:]/(hfBF.Gradients().grad_nuc()+ge[4])[1,2])
plt.scatter(range(1,8),abs(np.asarray(aaff_predict)-(hfBF.Gradients().grad_nuc()+gBF)[1,2])[1:]/(hfBF.Gradients().grad_nuc()+ge[4])[1,2])
plt.yscale("log")
plt.axhline(0,ls='--',color='black')

In [ ]:
fd_predict=[]
fd_predict.append((hfBF.Gradients().grad_nuc()+ge[4])[1,2])
fd_predict.append(fd_predict[0]+np.einsum('i,ijk->jk',fd1_8,(np.asarray(ge)))[1,2]/dl)
fd_predict.append(fd_predict[1]+np.einsum('i,ijk->jk',fd2_8,(np.asarray(ge)))[1,2]/dl**2/2)
fd_predict.append(fd_predict[2]+np.einsum('i,ijk->jk',fd3_6,(np.asarray(ge)))[1,2]/dl**3/6)
fd_predict.append(fd_predict[3]+np.einsum('i,ijk->jk',fd4_4,(np.asarray(ge)))[1,2]/dl**4/24 )
fd_predict.append(fd_predict[4]+np.einsum('i,ijk->jk',fd5_4,(np.asarray(ge)))[1,2]/dl**5/120)
fd_predict.append(fd_predict[5]+np.einsum('i,ijk->jk',fd6_2,(np.asarray(ge)))[1,2]/dl**6/720)
np.asarray(fd_predict),np.asarray(fd_predict)-(hfBF.Gradients().grad_nuc()+gBF)[1,2]

In [ ]:
plt.plot(range(1,7),abs(np.asarray(fd_predict)-(hfBF.Gradients().grad_nuc()+gBF)[1,2])[1:]/(hfBF.Gradients().grad_nuc()+ge[4])[1,2])
plt.scatter(range(1,7),abs(np.asarray(fd_predict)-(hfBF.Gradients().grad_nuc()+gBF)[1,2])[1:]/(hfBF.Gradients().grad_nuc()+ge[4])[1,2])
plt.yscale("log")